In [84]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
import os
import pinecone
from langchain.vectorstores import Pinecone
from loguru import logger

dotenv_path = "../secrets.env"
load_dotenv(dotenv_path)

True

In [83]:
INDEX_NAME = "dating-plan-gpt-index"
DIMENSION = 1536

In [ ]:
pinecone.delete_index('dating-plan-gpt-index')

In [123]:
loader = CSVLoader(file_path="../data/reviews.csv")
documents = loader.load()
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0.5, model="gpt-4")
pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.environ.get("PINECONE_API_ENV"),  # next to api key in console
)
docsearch = Pinecone.from_existing_index(index_name=INDEX_NAME, embedding=embeddings)



In [125]:

from langchain.prompts import PromptTemplate
prompt_template = """Act as a date planner and generate a creative and flexible dating plan based on the context and requirements provided. 
Requirements: Your plan should be adaptable to various user constraints such as time, preferences, and other specific requirements. 
Ensure your responses are diverse and not limited to a fixed number of locations or repetitive suggestions. 
Use your creativity to provide unique and engaging dating plans for each query. 
Make sure the timing of the activities in the plan makes sense; for longer available hours, suggest more than one item in the plan. 
As you currently do not have the ability to remember past conversations, ensure each response is tailored to the specific query at hand. 
Do not include "overall rating" and "category" when giving the user your final answer. 
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
When coming up with locations to suggest, you must use name field, which is the name of the restaruant/location/mall/museum. Do not use imaginary or made-up names.
Also, Take into account of the caption, category and opening_hours field.
Format the response in time-table like manner, and explain why you chose the locations you chose. Make them engaging and enticing.
Question: {question}

Answer: """

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [126]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 10}),
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True,
)

In [127]:
query = "I am available from 5pm to 9pm. Help me plan a date to a museum and japanese restaurant."
# print(qa.run(query))
result = qa({"query": query})


In [128]:
print(result["result"])

Sure, based on your preferences and the available time, here's a plan for your date:

5:00 pm - 7:00 pm: Start your evening at the National Museum of Singapore. This museum is known for its well-designed facilities and innovative exhibits. You can spend a good two hours exploring the various exhibits, which include a unique showcase of stories from low-income families about their relationship with food. It's not just a museum visit, but an immersive cultural experience. 

7:00 pm - 7:30 pm: Take a leisurely stroll or a short drive from the National Museum of Singapore to your dinner location. This transition time will give you and your date a chance to discuss the exhibits you've just seen and build up an appetite for dinner.

7:30 pm - 9:00 pm: Conclude your date with a delightful dining experience at KOMA Singapore. This Asian fusion restaurant is known for its personalized service, majestic oriental decorations, and delicious dishes. The staff even addresses guests by their names, a